In [1]:
# Problem 2 - Sports Team Optimization
# You are in charge of assembling a sports team with a budget constraint. You wish to pick
# the team that gives you the best chance of winning (by scoring the most points). Use the
# provided data to create a team picking system that optimizes for expected points,
# constrained by budget (B – nonnegative float), and number of players on a team (lower
# bound M, upper bound N – nonnegative integers).
# More formally:
# Maximize:
#  - Expected points
# Subject to:
#  - Total cost (sum of salaries paid) <= B
#  - Number of players chosen <= N
#  - Number of players chosen >= M
# Output: Work in a Jupyter notebook, R markdown file, or script detailing your process

# part 2 of problem 2 - 
# introduce the constraint that a team can only comprise of X% of each position

In [2]:
# importing the initial libraries/packages required for reading in the data and performing based transformations

%matplotlib notebook
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns # rich visualizations

# showing the folder structure/path to raw data
# import os
# for dirname, _, filenames in os.walk('/Users/alexanderblaies/Desktop'):
    # for filename in filenames:
       # print(os.path.join(dirname, filename))

In [3]:
# initial inspection
sports_df = pd.read_csv('/Users/alexanderblaies/Desktop/sports_data.csv')
sports_df.head(10)

,Unnamed: 0,player_id,position,salary,expected_points
0,0,412,A,11900,41.5
1,1,239,B,11700,78.6
2,2,2,B,11500,51.4
3,3,240,C,10400,68.8
4,4,241,A,10000,46.1
5,5,3,B,9900,53.1
6,6,413,B,9400,53.0
7,7,414,A,9400,20.4
8,8,5,A,9300,61.3
9,9,11,C,9200,33.1


In [4]:
# dropping the "Unnamed: 0" column; looks good
sports_df.rename(columns = {'Unnamed: 0':'Index'}, inplace = True)
sports_df.head(5)

,Index,player_id,position,salary,expected_points
0,0,412,A,11900,41.5
1,1,239,B,11700,78.6
2,2,2,B,11500,51.4
3,3,240,C,10400,68.8
4,4,241,A,10000,46.1


In [5]:
# checking the data types to ensure compatibility
sports_df.dtypes

Index                int64
player_id            int64
position            object
salary               int64
expected_points    float64
dtype: object

In [21]:
print(sports_df.position.unique())

['A' 'B' 'C']


In [6]:
# checking the summary statistics
sports_df.describe()

,Index,player_id,salary,expected_points
count,219.000000,219.000000,219.000000,219.000000
mean,109.000000,226.936073,5205.022831,21.464384
std,63.364028,157.768253,1873.235530,14.123291
min,0.000000,2.000000,3500.000000,0.000000
25%,54.500000,79.500000,3700.000000,11.750000
50%,109.000000,228.000000,4500.000000,19.700000
75%,163.500000,368.500000,6400.000000,28.900000
max,218.000000,511.000000,11900.000000,78.600000


In [7]:
# just double checking to ensure there are no null values
sports_df.isnull().any()

Index              False
player_id          False
position           False
salary             False
expected_points    False
dtype: bool

In [8]:
# let's go ahead and build the optimization model using the following variables; EDA would be somewhat valuable here, but
# it is not required to build the model 
# maximize P (points)
# constraints: B (budget - summation of salaries), M (lower bound of players available), N (upper bound of players available)
B_val = float(input("Enter your value for total budget available: "))
M_val = int(input("Enter your value for the lower bound of players available: "))
N_val = int(input("Enter your value for the upper bound of players available: "))


Enter your value for total budget available: 99423
Enter your value for the lower bound of players available: 11
Enter your value for the upper bound of players available: 11


In [9]:
# now, I could be a complex algo from scratch, but for the sake of time, I'll use libraries that are available
# that's where the "pulp" library comes into play
# importing the solver/optimizer
import pulp
from pulp import *

# initializing an instance of the solver:
model = LpProblem('Maximize Points', sense= LpMaximize)

# defining the decision variables (I can only maximize points with a given # of players; need to determine which
# players get selected by creating a binary option (0 = not selected, 1 = selected))
decision_variables = []
for rownum, row in sports_df.iterrows():
    variable = str('x' + str(row['Index']))
    variable = pulp.LpVariable(str(variable), lowBound = 0, upBound = 1, cat = 'Integer')
    decision_variables.append(variable)
print("total # of possible players to select:", str(len(decision_variables)))


total # of possible players to select: 219


/Users/alexanderblaies/opt/anaconda3/lib/python3.8/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [10]:
# defining the objective function: essentially summation of the expected_points given constraint criteria (budget, min
# and max available players to select)

# initializing a variable to hold the summation
total_expected_points = ""
# looping through the dataframe rows and matching based on index; evaluating expected points and summing where selection 
# equals 1 based on the solver
for rownum, row in sports_df.iterrows():
    for i, decision in enumerate(decision_variables):
        if i == rownum:
            formula = row['expected_points']*decision # value corresponds to a 1 or 0
            total_expected_points += formula

model += total_expected_points           

In [11]:
# defining the constraints
min_players = M_val
max_players = N_val
max_budget = B_val

min_players_selected = ""
for rownum, row in sports_df.iterrows():
    for i, decision in enumerate(decision_variables):
        if i == rownum:
            formula = decision
            min_players_selected += decision
            
model += (min_players_selected >= min_players)

max_players_selected = ""
for rownum, row in sports_df.iterrows():
    for i, decision in enumerate(decision_variables):
        if i == rownum:
            formula = decision
            max_players_selected += decision
            
model += (max_players_selected <= max_players)

budget_sum = ""
for rownum, row in sports_df.iterrows():
    for i, decision in enumerate(decision_variables):
        if i == rownum:
            formula = row['salary']*decision
            budget_sum += formula
            
model += (budget_sum <= max_budget)

In [12]:
# The final format of the problem formulated is written out into a .lp file. 
# This will list the objective function, the decision variables and the constraints imposed on the problem.

# print(model)
model.writeLP('Roster_Team_Build')

[x0,
 x1,
 x10,
 x100,
 x101,
 x102,
 x103,
 x104,
 x105,
 x106,
 x107,
 x108,
 x109,
 x11,
 x110,
 x111,
 x112,
 x113,
 x114,
 x115,
 x116,
 x117,
 x118,
 x119,
 x12,
 x120,
 x121,
 x122,
 x123,
 x124,
 x125,
 x126,
 x127,
 x128,
 x129,
 x13,
 x130,
 x131,
 x132,
 x133,
 x134,
 x135,
 x136,
 x137,
 x138,
 x139,
 x14,
 x140,
 x141,
 x142,
 x143,
 x144,
 x145,
 x146,
 x147,
 x148,
 x149,
 x15,
 x150,
 x151,
 x152,
 x153,
 x154,
 x155,
 x156,
 x157,
 x158,
 x159,
 x16,
 x160,
 x161,
 x162,
 x163,
 x164,
 x165,
 x166,
 x167,
 x168,
 x169,
 x17,
 x170,
 x171,
 x172,
 x173,
 x174,
 x175,
 x176,
 x177,
 x178,
 x179,
 x18,
 x180,
 x181,
 x182,
 x183,
 x184,
 x185,
 x186,
 x187,
 x188,
 x189,
 x19,
 x190,
 x191,
 x192,
 x193,
 x194,
 x195,
 x196,
 x197,
 x198,
 x199,
 x2,
 x20,
 x200,
 x201,
 x202,
 x203,
 x204,
 x205,
 x206,
 x207,
 x208,
 x209,
 x21,
 x210,
 x211,
 x212,
 x213,
 x214,
 x215,
 x216,
 x217,
 x218,
 x22,
 x23,
 x24,
 x25,
 x26,
 x27,
 x28,
 x29,
 x3,
 x30,
 x31,
 x32,
 x33,
 x3

In [13]:
# solving the model 
solution = model.solve()
 

In [14]:
# printing the variables and optimal solution based on user input:
# status: 1 - optimal, 2 - solved, 3 - infeasible, 4 - unbounded
print("Status:", LpStatus[model.status])
print("Optimal Solution", value(model.objective), "expected points")
print("Individual decision_variables: ")
for v in model.variables():
    print(v.name, "=", v.varValue)


Status: Optimal
Optimal Solution 620.4000000000001 expected points
Individual decision_variables: 
x0 = 0.0
x1 = 1.0
x10 = 0.0
x100 = 0.0
x101 = 0.0
x102 = 0.0
x103 = 0.0
x104 = 0.0
x105 = 0.0
x106 = 0.0
x107 = 0.0
x108 = 0.0
x109 = 0.0
x11 = 1.0
x110 = 0.0
x111 = 0.0
x112 = 0.0
x113 = 0.0
x114 = 0.0
x115 = 0.0
x116 = 0.0
x117 = 0.0
x118 = 0.0
x119 = 0.0
x12 = 0.0
x120 = 0.0
x121 = 0.0
x122 = 0.0
x123 = 0.0
x124 = 0.0
x125 = 0.0
x126 = 0.0
x127 = 0.0
x128 = 0.0
x129 = 0.0
x13 = 0.0
x130 = 0.0
x131 = 0.0
x132 = 0.0
x133 = 0.0
x134 = 0.0
x135 = 0.0
x136 = 0.0
x137 = 0.0
x138 = 0.0
x139 = 0.0
x14 = 0.0
x140 = 0.0
x141 = 0.0
x142 = 0.0
x143 = 0.0
x144 = 0.0
x145 = 0.0
x146 = 1.0
x147 = 0.0
x148 = 0.0
x149 = 0.0
x15 = 0.0
x150 = 0.0
x151 = 0.0
x152 = 0.0
x153 = 0.0
x154 = 0.0
x155 = 0.0
x156 = 0.0
x157 = 0.0
x158 = 0.0
x159 = 0.0
x16 = 0.0
x160 = 0.0
x161 = 0.0
x162 = 0.0
x163 = 0.0
x164 = 0.0
x165 = 0.0
x166 = 0.0
x167 = 0.0
x168 = 0.0
x169 = 0.0
x17 = 0.0
x170 = 0.0
x171 = 0.0
x172 = 0.0


In [15]:
# looking at the variable names (Index) that were selected by the solver
variable_names = []
variable_values = []
for v in model.variables():
    variable_names.append(v.name)
    variable_values.append(v.varValue)
    
opt_df = pd.DataFrame({"Index":variable_names,"Value":variable_values}).sort_values('Value', ascending = False)
filter_ = opt_df['Value']==1.0
opt_df = opt_df[filter_].set_index('Index')
opt_df.head(10)

,Value
Index,
x2,1.0
x11,1.0
x146,1.0
x45,1.0
x6,1.0
x3,1.0
x28,1.0
x23,1.0
x8,1.0


In [16]:
# referencing the original dataset to see which players were chosen along with their respective data
sports_df['reference_'] = sports_df.Index.apply(lambda x: 'x' + str(x))
sports_df = sports_df.set_index('reference_')
sports_df.head(5)

,Index,player_id,position,salary,expected_points
reference_,,,,,
x0,0,412,A,11900,41.5
x1,1,239,B,11700,78.6
x2,2,2,B,11500,51.4
x3,3,240,C,10400,68.8
x4,4,241,A,10000,46.1


In [19]:
# joining the two to show players selected and sorted by expected points
final_result = sports_df.join(opt_df, how='inner')
final_result = final_result[['player_id','position','salary','expected_points']].reset_index()
final_result.rename(columns={"index":"variable_reference"}, inplace = True)


In [20]:
final_result

,variable_reference,player_id,position,salary,expected_points
0,x1,239,B,11700,78.6
1,x2,2,B,11500,51.4
2,x3,240,C,10400,68.8
3,x5,3,B,9900,53.1
4,x6,413,B,9400,53.0
5,x8,5,A,9300,61.3
6,x11,7,A,8900,51.3
7,x23,17,A,8000,51.5
8,x28,18,C,7700,48.7
9,x45,258,A,6700,55.1
